In [109]:
!ls ../../evaluations/hate_speech/task_b/

bert-hier.json			  beto.json
bert.json			  robertuito-combi.json
beto-combi.json			  robertuito-hierarchical-gamma-0.1.json
beto-hierarchical-gamma-0.1.json  robertuito-hierarchical-gamma-0.json
beto-hierarchical-gamma-0.json	  robertuito.json


In [107]:
import json
import pandas as pd

evaluations = {

}

for model_name, path in [
    ('beto', '../../evaluations/hate_speech/task_b/beto.json'),
    ('beto-hier', '../../evaluations/hate_speech/task_b/beto-hierarchical-gamma-0.1.json'),
    #('beto-hier-zero', '../../evaluations/hate_speech/task_b/beto-hierarchical-gamma-0.json'),
    ('beto-combi', '../../evaluations/hate_speech/task_b/beto-combi.json'),
    ('robertuito', '../../evaluations/hate_speech/task_b/robertuito.json'),
    ('robertuito-hier', '../../evaluations/hate_speech/task_b/robertuito-hierarchical-gamma-0.1.json'),
    #('robertuito-hier-zero', '../../evaluations/hate_speech/task_b/robertuito-hierarchical-gamma-0.json'),
    ('robertuito-combi', '../../evaluations/hate_speech/task_b/robertuito-combi.json'),
    ]:
    with open(path) as f:
        evaluations[model_name] = json.load(f)


for key, evals in evaluations.items():
    print(f"We have {len(evals['evaluations']['hate_speech'])} {key} evaluations")



dfs = []
for model_name, model_results in evaluations.items():
    model_evaluations = model_results["evaluations"]["hate_speech"]
    
    if not model_evaluations:
        continue
    
    df = pd.DataFrame(model_evaluations)

    df.columns = [x.split("_", 1)[1] if "_" in x else x for x in df.columns]
    
    mean_df = pd.DataFrame({
        f"{model_name} mean": df.mean(), 
        f"{model_name} std": df.std()
    })
    dfs.append(mean_df)

result_df = pd.concat(dfs, axis=1)
index = [
    'hs_f1', 'hs_precision', 'hs_recall', 'tr_f1',
    'ag_f1', 'macro_hs_f1_score',
    'emr_no_gating', 'emr', 
    'macro_f1',
]
result_df.loc[index, [c for c in result_df.columns if "std" not in c]]

We have 10 beto evaluations
We have 10 beto-hier evaluations
We have 10 beto-combi evaluations
We have 9 bert evaluations
We have 6 bert-hier evaluations


,hs_precision,hs_recall,hs_f1,macro_hs_f1_score,tr_f1,ag_f1,emr_no_gating,emr,macro_f1
beto mean,0.713,0.778,0.741,0.771,0.765,0.688,0.684,0.685,0.731
beto-hier mean,0.744,0.728,0.735,0.776,0.758,0.674,0.674,0.703,0.722
beto-combi mean,0.726,0.762,0.742,0.776,0.763,0.668,0.698,0.698,0.724
bert mean,0.479,0.954,0.638,0.513,0.600,0.442,0.380,0.381,0.560
bert-hier mean,0.479,0.961,0.639,0.509,0.587,0.445,0.363,0.378,0.557


In [88]:
import scipy.stats

metric = "eval_emr"

combi = {
    "f1": [evaluation["eval_macro_f1"] for evaluation in evaluations["beto-combi"]["evaluations"]["hate_speech"]],
    "emr": [evaluation["eval_emr"] for evaluation in evaluations["beto-combi"]["evaluations"]["hate_speech"]]
}

hier = {
    "f1": [evaluation["eval_macro_f1"] for evaluation in evaluations["beto-hier"]["evaluations"]["hate_speech"]],
    "emr": [evaluation["eval_emr"] for evaluation in evaluations["beto-hier"]["evaluations"]["hate_speech"]]
}


standard = {
    "f1": [evaluation["eval_macro_f1"] for evaluation in evaluations["beto"]["evaluations"]["hate_speech"]],
    "emr": [evaluation["eval_emr"] for evaluation in evaluations["beto"]["evaluations"]["hate_speech"]]
}


Corremos un test de Kruskal para ver si la diferencia entre las medias es significativa


In [95]:
scipy.stats.kruskal(standard["emr"], hier["emr"], combi["emr"])

KruskalResult(statistic=3.4920080142475607, pvalue=0.17446973214802494)

In [96]:
scipy.stats.kruskal(standard["f1"], hier["f1"], combi["f1"])

KruskalResult(statistic=3.6567741935483866, pvalue=0.16067250810281022)

In [91]:
scipy.stats.mannwhitneyu(standard["emr"], hier["emr"], )

MannwhitneyuResult(statistic=26.5, pvalue=0.08186862373064437)

No lo es

In [44]:
import scipy.stats

combi_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito-combi"]["evaluations"]["hate_speech"]]
hier_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito-hier"]["evaluations"]["hate_speech"]]
standard_emr = [evaluation["eval_emr"] for evaluation in evaluations["robertuito"]["evaluations"]["hate_speech"]]

scipy.stats.kruskal(standard_emr, hier_emr, combi_emr)

KruskalResult(statistic=10.860138146167559, pvalue=0.004382793059181017)

En el caso de robertuito sí! wtf